In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [210]:
# ls

In [3]:
tickets = pd.read_csv('anonymized_hubspot_tickets.csv')
deals = pd.read_csv('anonymized_hubspot_deals.csv')
companies = pd.read_csv('anonymized_hubspot_companies.csv')

/var/folders/vs/xwzq4hsj1lz9tz8k9jz_tw540000gn/T/ipykernel_28763/2095908849.py:3: DtypeWarning: Columns (24,25,28,32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  companies = pd.read_csv('anonymized_hubspot_companies.csv')


In [4]:
len(deals)

593

In [5]:
len(companies)

19851

In [11]:
len(tickets)

79

In [13]:
pd.set_option('display.max_columns', None)
tickets.head(n = 2)

,Create date,Ticket Tags,Stage Date - Project Launch,Associated Contact,Category,Target Launch Date,Kickoff Call,Close date,Pipeline,Stage Date - Project Initiation,Stage Date - Planning Phase,Ticket status,Stage Date - Execution,Time to first agent email reply (HH:mm:ss),Last CES survey rating,Priority,Associated Company,Who will be creating the Syms?,Stage Date - Closure Phase,Stage Date - Monitoring and Control Phase,Associated Company (Primary),Response time (HH:mm:ss),Ticket ID,Time to close (HH:mm:ss),Associated Deal,Library index approved,Training: General Overview,Latest Milestone,Training: Deployment/User Management Training,Requirements for the Trial,Training: Sym Building 101,Last modified date,Type,Trial End Date,Training: Sym Building 201,Stage Date - Converted Won,Latest Milestone Update Date,Ticket name,Trial Overview,Trial Start Date,1st Syms presented for review,Project Launch Day,Training: Reporting,Construction of 1st Sym begun,Trial Required,Was the sym QAed?
0,2023-12-18 15:02,NaN,NaN,Contact_b6d930b5,NaN,NaN,2023-12-27,2024-01-29 11:50,Project Success Plan,2023-12-18,NaN,Converted-Won,2024-01-02,NaN,NaN,NaN,Company_67d87cec,NaN,2024-01-29,NaN,Company_67d87cec,NaN,2193923207,1004:48:10,Deal_67d87cec,No,NaN,NaN,NaN,Coaching,NaN,2025-02-14 13:39,NaN,NaN,NaN,2024-02-06,NaN,Ticket_4baf7886,NaN,NaN,NaN,NaN,NaN,2023-12-27,NaN,NaN
1,2024-10-30 12:31,NaN,NaN,Contact_aa84b2e1,NaN,NaN,NaN,2025-02-10 11:18,Project Success Plan,NaN,NaN,Lost,NaN,49:24:00,NaN,NaN,Company_7588d0d5,NaN,NaN,NaN,Company_67d87cec,49:24:00,15899166073,2471:47:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-14 13:39,NaN,2024-11-07,NaN,NaN,NaN,Ticket_88dda9e1,NaN,2024-11-05,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
### remove columns with more than 90% missing values
tickets_col_remove = tickets.loc[:, tickets.isnull().mean() > 0.9].columns
tickets = tickets.loc[:, tickets.isnull().mean() < 0.9]

### To connect tickets data with other datasets:
- Associated Company (Primary) --> links to companies dataset (doesn't work very well) 
- Associated Deal --> links to deals dataset
- Ticket ID --> links to companies dataset via CompanyToTickets mapping

In [215]:
# tickets['Associated Company (Primary)']

In [40]:
pd.set_option('display.max_columns', None)
companies.head(n = 2)

,Parent Company,CCaaS,Annual Revenue,Target Account,Associated Contact,Number of Form Submissions,Total Agents,Web Technologies,Close Date,# of Agents Total,Number of times contacted,Contact with Primary Company,ICP Fit Level,Revenue range,# of Agents Contracted,Record ID,Time Zone,Primary Industry,Number of Pageviews,Primary Company,Year Founded,ICP,Industry group,Segmentation,LMS System,SymTrain Use Cases,Associated Company,BPO,SSO Application,State/Region,SymTrain Product,Contract End Date,SSO Implemented?,Consolidated Industry,Type,Number of Employees,BPO Program,Primary Sub-Industry,Number of Sessions,WFM,Country/Region,Industry,Create Date,Company name,Last Modified Date,BPO Program Tier
0,NaN,NaN,1.000000e+09,NaN,Contact_ef780380,0.0,NaN,Route 53,NaN,NaN,5.0,Company_ef780380,NaN,NaN,NaN,25246439235,America/New_York,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OH,NaN,NaN,NaN,(No value),NaN,5000.0,NaN,NaN,0.0,NaN,United States,HVAC and plumbing supply,2024-10-30 10:51,Company_4fc73a2a,2025-02-14 13:39,NaN
1,NaN,NaN,NaN,NaN,Contact_93373ba5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22987081774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(No value),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pest Control,2024-09-12 18:30,Company_f3f7e884,2025-02-14 13:39,NaN


# Merge the tickets dataframe and the companies dataset
### Approach 1: Use the Associated Company (Primary) from tickets and Company name from companies to merge the 2 dataframes, only 48 overlaps

In [221]:
### only 48 out of 79 overlaps between company ids from ticket and company dataset
len(set(tickets['Associated Company (Primary)']).intersection(companies['Company name']))

48

### Approach 2: Use the mappings from the mappings.json file

In [158]:
### read in the mapping file
import json

with open('mappings.json', 'r') as f:
    mappings = json.load(f)

In [225]:
mappings.keys()

dict_keys(['CompanyToDeals', 'CompanyToTickets', 'ParentChildRelationships', 'TicketToDeal'])

- `CompanyToDeals`: contains the mappings from company ID to Deals ID
- `CompanyToTickets`: contains the mappings from company ID to Tickets ID
- `TicketToDeal`: contains the mappings from Ticket ID to Deal ID 

In [235]:
list(mappings['CompanyToTickets'].items())[:2]
### key is the company id and value is a list of tickets ID, same company can have multiple tickets ID

[('9366128719', ['2227268264']),
 ('9355871862',
  ['1153128060',
   '1184439975',
   '1255495303',
   '1258208860',
   '1265785475',
   '1271135644',
   '1294735265',
   '1296806165',
   '1305598867',
   '1317059345',
   '1353741690',
   '1377812063',
   '1380677892',
   '1428597842',
   '1436802687',
   '1444251475',
   '1486045610',
   '1486225212',
   '1490751498',
   '1501062400',
   '1525208184',
   '1526946139',
   '1528572955',
   '1531938692',
   '1575993731',
   '1578772008',
   '1595350578',
   '1597266872',
   '1597391905',
   '1601174888',
   '1602159132',
   '1606567201',
   '1606585647',
   '1606960707',
   '1608271468',
   '1608744106',
   '1609058103',
   '1609091896',
   '1609096794',
   '1609111093',
   '1609765740',
   '1609979182',
   '1610373386',
   '1610467704',
   '1610471190',
   '1610477343',
   '1610876991',
   '1611925122',
   '1613205790',
   '1615039154',
   '1615155028',
   '1615740539',
   '1616085107',
   '1616912930',
   '1616927862',
   '1617059116',

### Now, create a dictionary based on mappings['CompanyToTickets'] where the keys are ticket IDs and the values are company IDs. This will give us a one-to-one mapping between ticket IDs and company IDs, which we can later use to add a new company_id column to the ticket DataFrame.

In [238]:
# revert the company-ticket mapping
ticket_to_company = {}

for company_id, ticket_list in mappings["CompanyToTickets"].items():
    for ticket_id in ticket_list:
        ticket_to_company[ticket_id] = company_id

In [180]:
# ticket_to_company

In [170]:
### convert the ticket id column into string type
tickets['Ticket ID'] = tickets['Ticket ID'].astype(str)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [174]:
### create a company ID column in the tickets dataset based on 'ticket_to_company' dictionary
tickets['Company ID'] = tickets['Ticket ID'].map(ticket_to_company)

In [192]:
### filter the company dataset and only keep columns with less than 90% missing values, you can also try your own threshold
companies = companies.loc[:, companies.isnull().mean() < 0.9]

In [204]:
### convert the integer type into string type for later mapping
companies['Record ID'] = companies['Record ID'].astype(str)

/var/folders/vs/xwzq4hsj1lz9tz8k9jz_tw540000gn/T/ipykernel_28763/4261195776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies['Record ID'] = companies['Record ID'].astype(str)


In [208]:
### you can merge the companys dataframe and the ticket dataframe
### here I did left merge, so only 79 rows are kept since tickets data only contains 79 rows
merged_tickets = pd.merge(left = tickets, right= companies, left_on= 'Company ID', right_on = 'Record ID', how = 'left')

### You can follow a similar pattern to merge the tickets and deals datasets, or choose to merge only the companies and deals datasets, since the tickets dataset contains only 79 records.

# Problems to solve in the Final Report and Posisble Solutions:



In [248]:
import pandas as pd

# Define the left column (objectives) and right column (model suggestions)
objectives = [
    "Segment prospective customers based on company characteristics and provide tailored outreach strategies",
    "Guide sales progression through pipeline stages with specific actions for each stage",
    "Predict deal outcomes based on company profile and deal characteristics",
    "Optimize implementation handoffs between sales and customer success",
    "Recommend resource allocation based on customer segment and complexity",
    "Present insights through an interactive dashboard for sales team usage",
    "Process data through an ETL pipeline to keep recommendations current",
    "Deploy as a containerized application for consistent environment setup"
]

model_suggestions = [
    "Clustering (KMeans, DBSCAN), EDA",
    'Look into the ticket data and the deals data, is all steps always needed for a successful deal? Is training always needed for upsell and new customers? ',
    "Classification models (Logistic Regression, Random Forest, XGBoost, Neural Network for Binary classification)",
    " ",
    "complexity can be measured by total tickets needed for each deal, average resolution time needed, etc, does different segments of customers have the same implementation complexity?  Does different segments of customers requires same actions?",
    "Streamlit",
    "ETL frameworks (Pandas pipelines), data cleaning scripts",
    "Docker, Docker Compose, Kubernetes for containerization and deployment"
]

# Create the dataframe
df = pd.DataFrame({
    "Project Objective": objectives,
    "Suggested Models / Tools": model_suggestions
})




In [250]:
df

,Project Objective,Suggested Models / Tools
0,Segment prospective customers based on company...,"Clustering (KMeans, DBSCAN), EDA"
1,Guide sales progression through pipeline stage...,"Look into the ticket data and the deals data, ..."
2,Predict deal outcomes based on company profile...,"Classification models (Logistic Regression, Ra..."
3,Optimize implementation handoffs between sales...,
4,Recommend resource allocation based on custome...,complexity can be measured by total tickets ne...
5,Present insights through an interactive dashbo...,Streamlit
6,Process data through an ETL pipeline to keep r...,"ETL frameworks (Pandas pipelines), data cleani..."
7,Deploy as a containerized application for cons...,"Docker, Docker Compose, Kubernetes for contain..."


### Possible Q&A



### Q1: What does it mean by ‘customer segment and complexity’ in the requirement *“Recommend resource allocation based on customer segment and complexity”*? Also, what are the actions to take at each stage — aren’t those actions fixed?

**A1:**  
Customer segment and implementation complexity can be evaluated primarily through the **ticket data**. Some useful indicators include:

- Number of support tickets generated per implementation (more tickets may indicate higher complexity)
- Average resolution time for implementation-related issues
- Whether SSO (Single Sign-On) was required and whether it accelerated or delayed implementation

You can quantify **implementation complexity** by engineering a feature that aggregates these signals.


### Q2: What are the actions to take at each stage — aren’t those actions fixed?

**A:**  
Yes, the standard actions for each stage are typically fixed. However, the ticket and deal data may reveal valuable insights that can help **adapt or skip certain stages** under specific conditions. For example:

- If the deal is an **upsell** to an existing customer, the **training stage** may not be necessary, as the customer is already familiar with the product.
- What does historical data show? Have some successful deals **skipped onboarding or negotiation** and still closed faster or more successfully?






# Final Report Format

1. Title Page (project title, course name, group member name and VUID)
2. Introduction
3. Data Overview & Exploration
4. Data Processing
5. Model Deployment
6. Dashboard & Visualization
7. Deployment & Technical Implementation
8. Business Insights & Recommendations
9. Conclusion & Future Work
10. References (if applicable) 
